In [7]:
import requests
import json
import pandas as pd
import numpy as np
import datetime

In [14]:
def get_urls(start_epoch, end_epoch):
    curr_start = start_epoch
    one_day = 86400
    ret = []
    # can only do 500 results
    while curr_start < end_epoch:
        temp_end = min(end_epoch, curr_start + (one_day * 300))
        url = f"https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start={curr_start}&end={temp_end}&period=86400"
        ret.append(url)
        curr_start = temp_end
    return ret

# from 2015 to 2020 since USDT came out in 2014

In [15]:
urls = get_urls(int(datetime.datetime(2016, 1, 1, 0, 0, 0).strftime("%s")), int(datetime.datetime(2021, 12, 31, 23, 59, 59).strftime("%s")))   # 6 years from 2016 to 2021

df = None
for url in urls:
    req = requests.get(url)
    json_resp = json.loads(req.text)
    if df is None:
        df = pd.json_normalize(json_resp)
    else:
        df = pd.concat([df, pd.json_normalize(json_resp)])

df

,date,high,low,open,close,volume,quoteVolume,weightedAverage
0,1451692800000,0.95,0.95,0.95,0.95,5.603982980000000000,5.363009890000000000,0.956999670000000000
1,1451779200000,0.97,0.95,0.95,0.96,482.036490860000000000,465.891873470000000000,0.966507470000000000
2,1451865600000,0.96,0.93,0.96,0.96,132.451743440000000000,126.214022850000000000,0.952905710000000000
3,1451952000000,0.95,0.93,0.93,0.93,219.947631600000000000,206.680281620000000000,0.939679500000000000
4,1452038400000,0.93,0.93,0.93,0.93,71.865247650000000000,67.445978190000000000,0.938506160000000000
...,...,...,...,...,...,...,...,...
88,1640649600000,4037.79,3760.75,4037.79,3792.28,3755.304233990000000000,14602542.723782990000000000,3888.511240070000000000
89,1640736000000,3826.43,3603.00,3793.87,3629.68,3140.234921020000000000,11735452.510536170000000000,3737.125662790000000000
90,1640822400000,3767.99,3586.22,3629.30,3709.31,2356.571465070000000000,8740104.915044660000000000,3708.822348310000000000
91,1640908800000,3821.62,3606.91,3709.22,3677.49,5668.366520440000000000,21249209.112394930000000000,3748.735907560000000000


In [16]:
df = df.astype({
    "high": np.float64,
    "low": np.float64,
    "open": np.float64,
    "close": np.float64,
    "volume": np.float64,
    "quoteVolume": np.float64,
    "weightedAverage": np.float64
})
df.dtypes

date                object
high               float64
low                float64
open               float64
close              float64
volume             float64
quoteVolume        float64
weightedAverage    float64
dtype: object

In [17]:
df

,date,high,low,open,close,volume,quoteVolume,weightedAverage
0,1451692800000,0.95,0.95,0.95,0.95,5.603983,5.363010e+00,0.957000
1,1451779200000,0.97,0.95,0.95,0.96,482.036491,4.658919e+02,0.966507
2,1451865600000,0.96,0.93,0.96,0.96,132.451743,1.262140e+02,0.952906
3,1451952000000,0.95,0.93,0.93,0.93,219.947632,2.066803e+02,0.939680
4,1452038400000,0.93,0.93,0.93,0.93,71.865248,6.744598e+01,0.938506
...,...,...,...,...,...,...,...,...
88,1640649600000,4037.79,3760.75,4037.79,3792.28,3755.304234,1.460254e+07,3888.511240
89,1640736000000,3826.43,3603.00,3793.87,3629.68,3140.234921,1.173545e+07,3737.125663
90,1640822400000,3767.99,3586.22,3629.30,3709.31,2356.571465,8.740105e+06,3708.822348
91,1640908800000,3821.62,3606.91,3709.22,3677.49,5668.366520,2.124921e+07,3748.735908


In [18]:
df['date'] = pd.to_datetime(df['date'], unit='ms')
df = df.drop_duplicates(subset=["date"])
df = df.set_index("date")
df.to_pickle("../../data/BTC_USDT_D.pkl")